In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [ ]:
data1=pd.read_csv("credit_record.csv")
data2=pd.read_csv("application_record.csv")

In [ ]:
data=pd.merge(data1,data2, on="ID")
data.head()

In [ ]:
data.info()

In [ ]:
print(data["NAME_INCOME_TYPE"].unique(),  data["NAME_EDUCATION_TYPE"].unique(),data["NAME_FAMILY_STATUS"].unique() ,data["NAME_HOUSING_TYPE"].unique())

In [ ]:
data=data.drop(columns="OCCUPATION_TYPE",axis=1)

In [ ]:
data

In [ ]:
import plotly.express as px
data_ge = data[data["CODE_GENDER"] == "F"]
data_ge = data_ge[data_ge["FLAG_OWN_CAR"] != "Y"]
data_ge = data_ge[data_ge["AMT_INCOME_TOTAL"] > 50000]

fig = px.scatter(data_ge, x="AMT_INCOME_TOTAL", y="DAYS_BIRTH", color="DAYS_BIRTH", size="AMT_INCOME_TOTAL",
                 hover_data=["FLAG_OWN_CAR", "CODE_GENDER"], size_max=30, title="FEMALE INCOMEvs. Days Away from birthday")
fig.show()

In [ ]:
data_ge = data_ge[data_ge["AMT_INCOME_TOTAL"] > 50000]
data_ge = data_ge[data_ge["CNT_FAM_MEMBERS"] > 1]

fig = px.scatter(data_ge, x="AMT_INCOME_TOTAL", y="CNT_FAM_MEMBERS", color="CNT_FAM_MEMBERS", size="AMT_INCOME_TOTAL",
                 hover_data=["CNT_FAM_MEMBERS", "CODE_GENDER"], size_max=50, title="INCOME vs. FAMILY SIZE")
fig.show()

In [ ]:
data_ge = data[data["CODE_GENDER"] == "M"]
data_ge = data_ge[data_ge["FLAG_OWN_CAR"] != "Y"]
data_ge = data_ge[data_ge["AMT_INCOME_TOTAL"] > 50000]

fig = px.scatter(data_ge, x="AMT_INCOME_TOTAL", y="DAYS_BIRTH", color="DAYS_BIRTH", size="AMT_INCOME_TOTAL",
                 hover_data=["FLAG_OWN_CAR", "CODE_GENDER"], size_max=30, title="MALE INCOME vs. AGE")
fig.show()

In [ ]:
data_ge = data_ge[data_ge["AMT_INCOME_TOTAL"] > 50000]
data_ge = data_ge[data_ge["CNT_FAM_MEMBERS"] > 1]

fig = px.scatter(data_ge, x="AMT_INCOME_TOTAL", y="CNT_FAM_MEMBERS", color="CNT_FAM_MEMBERS", size="AMT_INCOME_TOTAL",
                 hover_data=["CNT_FAM_MEMBERS", "CODE_GENDER"], size_max=50, title="INCOME vs. FAMILY SIZE")
fig.show()

In [ ]:
data["CODE_GENDER"] = data["CODE_GENDER"].apply(lambda x: 1 if x == "F" else 0)
data["FLAG_OWN_CAR"] = data["FLAG_OWN_CAR"].apply(lambda x: True if x == "Y" else False)
data["FLAG_OWN_REALTY"] = data["FLAG_OWN_REALTY"].apply(lambda x: True if x == "Y" else False)

# Encode education types
education_mapping = {
    "Incomplete higher": 1,
    "Higher education": 2,
    "Lower secondary": 3,
    "Secondary / secondary special": 4
}
data["NAME_EDUCATION_TYPE"] = data["NAME_EDUCATION_TYPE"].map(education_mapping).fillna(5).astype(int)
data= pd.get_dummies(data, columns = ["NAME_INCOME_TYPE", "NAME_FAMILY_STATUS","NAME_HOUSING_TYPE"])
data=data.drop(columns=["NAME_HOUSING_TYPE_With parents","NAME_INCOME_TYPE_Student","NAME_FAMILY_STATUS_Widow"])

In [ ]:
y = data["STATUS"]
data=data.drop(columns=["STATUS"])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
y = y.astype(str)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y) 
train_data,test_data,y_train,y_test=train_test_split(data,y,test_size=0.3,random_state=42,shuffle=True)

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_data,y_train)

In [ ]:
y_pred = clf.predict(test_data)
decision_accuracy = accuracy_score(y_test, y_pred)
decision_report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

In [ ]:
print(decision_accuracy)
print(decision_report)

# Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(train_data,y_train)
y_pred=clf.predict(test_data)
RF_accuracy = accuracy_score(y_test, y_pred)
RF_report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

# XGBOOST

In [ ]:
def convert(x):
    if x=="C":
        return 6
    elif x=="X":
        return 7
    else:
        return int(x)
y_train=y_train.apply(convert)
y_test=y_test.apply(convert) 
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(objective='multi:softmax', num_class=len(label_encoder.classes_), random_state=42)
xgb_clf.fit(train_data, y_train)
y_pred = xgb_clf.predict(test_data)
target_names = [str(cls) for cls in label_encoder.classes_]

# Calculate accuracy and generate classification report
xgb_accuracy = accuracy_score(y_test_encoded, y_pred)
xgb_report = classification_report(y_test_encoded, y_pred, target_names=target_names)

# Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()

# Train the model
lda_clf.fit(train_data, y_train)

# Make predictions on the test data
y_pred = lda_clf.predict(test_data)
print(np.unique(y_pred))

# Evaluate the model
lda_accuracy = accuracy_score(y_test, y_pred)
lda_report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)


In [ ]:
plt.plot(["Decision Tree","Random Forest", "XGBOOST", "Linear Discriminant Analysis"],[decision_accuracy,RF_accuracy,xgb_accuracy,lda_accuracy])